In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
house_raw_data = pd.read_csv("cal_housing_clean.csv")

In [3]:
house_raw_data.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
x_data = house_raw_data.drop("medianHouseValue",axis=1)

In [5]:
y = house_raw_data["medianHouseValue"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y, test_size=0.33, random_state=101)

In [55]:
X_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
16624,27.0,2863.0,711.0,930.0,533.0,2.6205
14848,19.0,3635.0,1078.0,3127.0,1098.0,1.3240
11081,34.0,2004.0,331.0,843.0,328.0,3.5900
930,23.0,3170.0,532.0,1446.0,515.0,4.4357
5889,40.0,3068.0,756.0,1190.0,695.0,3.5637


In [7]:
X_train.size

82968

In [8]:
X_test.size

40872

In [9]:
# normalize data which will be between 0 and 1
# ref) https://www.quora.com/What-is-the-meaning-of-min-max-normalization
minMaxScaler = MinMaxScaler()

In [10]:
# change scaler so it fits into data
minMaxScaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [11]:
scaled_X_train = pd.DataFrame(data=minMaxScaler.transform(X_train), columns=X_train.columns, index= X_train.index)

In [12]:
scaled_X_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
16624,0.509804,0.072766,0.110180,0.032455,0.087486,0.146246
14848,0.352941,0.092400,0.167132,0.109372,0.180398,0.056834
11081,0.647059,0.050918,0.051210,0.029409,0.053774,0.213107
930,0.431373,0.080574,0.082402,0.050520,0.084526,0.271431
5889,0.764706,0.077980,0.117163,0.041557,0.114126,0.211294


In [13]:
scaled_X_test = pd.DataFrame(data=minMaxScaler.transform(X_test), columns= X_test.columns, index= X_test.index)

In [14]:
house_raw_data.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [15]:
age = tf.feature_column.numeric_column(key="housingMedianAge")
num_rooms = tf.feature_column.numeric_column(key="totalRooms")
num_bed = tf.feature_column.numeric_column(key="totalBedrooms")
population = tf.feature_column.numeric_column(key="population")
households = tf.feature_column.numeric_column(key="households")
income = tf.feature_column.numeric_column(key="medianIncome")

In [73]:
feature_cols = [age, num_rooms, num_bed, population,households, income]

In [153]:
# instructor said he decided to have 6 units because we have 6 features
model = tf.estimator.DNNRegressor(feature_columns= feature_cols, hidden_units=[6,6,6,6,6] )
# model = tf.estimator.LinearRegressor(feature_columns= feature_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/9c/883h376s53500dcqng4b4nj00000gn/T/tmp_456mpre', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c41333cf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [165]:
input_func = tf.estimator.inputs.pandas_input_fn(x= scaled_X_train, y= y_train, batch_size=128 , num_epochs= 200, shuffle= True )

In [166]:
model.train(input_fn= input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/9c/883h376s53500dcqng4b4nj00000gn/T/tmp_456mpre/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 20000 into /var/folders/9c/883h376s53500dcqng4b4nj00000gn/T/tmp_456mpre/model.ckpt.
INFO:tensorflow:loss = 683091600000.0, step = 20001
INFO:tensorflow:global_step/sec: 218.916
INFO:tensorflow:loss = 786395200000.0, step = 20101 (0.461 sec)
INFO:tensorflow:global_step/sec: 298.829
INFO:tensorflow:loss = 1023785900000.0, step = 20201 (0.333 sec)
INFO:tensorflow:global_step/sec: 310.917
INFO:tensorflow:loss = 732920000000.0, step = 20301 (0.321 sec)
INFO:tensorflow:global_step/sec: 291.918
INFO:tensorflow:loss = 621169600000.0, step = 20401 (0.343 sec)
INFO:tensorflow:global_step/sec: 282.86
INFO:te

INFO:tensorflow:global_step/sec: 307.229
INFO:tensorflow:loss = 619003450000.0, step = 27401 (0.324 sec)
INFO:tensorflow:global_step/sec: 280.869
INFO:tensorflow:loss = 597349600000.0, step = 27501 (0.357 sec)
INFO:tensorflow:global_step/sec: 274.244
INFO:tensorflow:loss = 565769600000.0, step = 27601 (0.367 sec)
INFO:tensorflow:global_step/sec: 325.729
INFO:tensorflow:loss = 934396700000.0, step = 27701 (0.305 sec)
INFO:tensorflow:global_step/sec: 278.725
INFO:tensorflow:loss = 660342600000.0, step = 27801 (0.358 sec)
INFO:tensorflow:global_step/sec: 310.908
INFO:tensorflow:loss = 779393560000.0, step = 27901 (0.321 sec)
INFO:tensorflow:global_step/sec: 326.806
INFO:tensorflow:loss = 570644040000.0, step = 28001 (0.305 sec)
INFO:tensorflow:global_step/sec: 288.176
INFO:tensorflow:loss = 804436840000.0, step = 28101 (0.348 sec)
INFO:tensorflow:global_step/sec: 312.429
INFO:tensorflow:loss = 742481100000.0, step = 28201 (0.322 sec)
INFO:tensorflow:global_step/sec: 324.266
INFO:tensorflo

INFO:tensorflow:loss = 906769100000.0, step = 35201 (0.334 sec)
INFO:tensorflow:global_step/sec: 298.789
INFO:tensorflow:loss = 662771470000.0, step = 35301 (0.335 sec)
INFO:tensorflow:global_step/sec: 320.499
INFO:tensorflow:loss = 860183600000.0, step = 35401 (0.313 sec)
INFO:tensorflow:global_step/sec: 258.706
INFO:tensorflow:loss = 1125952300000.0, step = 35501 (0.388 sec)
INFO:tensorflow:global_step/sec: 300.953
INFO:tensorflow:loss = 732430660000.0, step = 35601 (0.330 sec)
INFO:tensorflow:global_step/sec: 303.301
INFO:tensorflow:loss = 740136700000.0, step = 35701 (0.333 sec)
INFO:tensorflow:global_step/sec: 272.686
INFO:tensorflow:loss = 514358900000.0, step = 35801 (0.364 sec)
INFO:tensorflow:global_step/sec: 269.538
INFO:tensorflow:loss = 631633200000.0, step = 35901 (0.370 sec)
INFO:tensorflow:global_step/sec: 221.303
INFO:tensorflow:loss = 625216600000.0, step = 36001 (0.453 sec)
INFO:tensorflow:global_step/sec: 286.942
INFO:tensorflow:loss = 780609900000.0, step = 36101 (0

In [167]:
# if i use scaled x data , i have use scaled test data
test_input_func = tf.estimator.inputs.pandas_input_fn(x= scaled_X_test,y= y_test , batch_size= 10, num_epochs= 1, shuffle= False)

In [168]:
test_results = model.evaluate(input_fn= test_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-13T17:20:44Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/9c/883h376s53500dcqng4b4nj00000gn/T/tmp_456mpre/model.ckpt-41607
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-13-17:20:46
INFO:tensorflow:Saving dict for global step 41607: average_loss = 5956340000.0, global_step = 41607, label/mean = 206802.39, loss = 59493530000.0, prediction/mean = 205763.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 41607: /var/folders/9c/883h376s53500dcqng4b4nj00000gn/T/tmp_456mpre/model.ckpt-41607


In [169]:
test_results

{'average_loss': 5956340000.0,
 'label/mean': 206802.39,
 'loss': 59493530000.0,
 'prediction/mean': 205763.0,
 'global_step': 41607}

In [170]:
# using same test data for predictions
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=scaled_X_test, y= y_test , batch_size= 10, num_epochs= 1, shuffle= False)

In [171]:
pred_obj = model.predict(input_fn=pred_input_func)

In [172]:
predictions = list(pred_obj)
predictions

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/9c/883h376s53500dcqng4b4nj00000gn/T/tmp_456mpre/model.ckpt-41607
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([296353.2], dtype=float32)},
 {'predictions': array([577026.2], dtype=float32)},
 {'predictions': array([155133.97], dtype=float32)},
 {'predictions': array([190505.69], dtype=float32)},
 {'predictions': array([467893.], dtype=float32)},
 {'predictions': array([180902.52], dtype=float32)},
 {'predictions': array([209806.83], dtype=float32)},
 {'predictions': array([184041.89], dtype=float32)},
 {'predictions': array([262412.22], dtype=float32)},
 {'predictions': array([321169.5], dtype=float32)},
 {'predictions': array([206685.86], dtype=float32)},
 {'predictions': array([207389.34], dtype=float32)},
 {'predictions': array([166740.03], dtype=float32)},
 {'predictions': array([159220.67], dtype=float32)},
 {'predictions': array([314788.75], dtype=float32)},
 {'predictions': array([192316.55], dtype=float32)},
 {'predictions': array([158084.98], dtype=float32)},
 {'predictions': array([205447.84], dtype=float32)},
 {'predictions': array([189035.58], dtype=float32)}

In [173]:
final_preds = []
for pred in predictions:
    final_preds.append(pred["predictions"])
    print(pred["predictions"])

[296353.2]
[577026.2]
[155133.97]
[190505.69]
[467893.]
[180902.52]
[209806.83]
[184041.89]
[262412.22]
[321169.5]
[206685.86]
[207389.34]
[166740.03]
[159220.67]
[314788.75]
[192316.55]
[158084.98]
[205447.84]
[189035.58]
[351582.44]
[176905.22]
[268676.9]
[218421.97]
[151917.95]
[149506.03]
[413199.2]
[148563.08]
[55072.55]
[115607.55]
[189951.11]
[263036.94]
[165857.22]
[216879.28]
[294503.1]
[179435.78]
[138088.9]
[143892.61]
[151408.28]
[176483.16]
[213189.56]
[184143.62]
[196648.64]
[185272.03]
[149411.9]
[255711.5]
[284284.]
[200781.9]
[125301.266]
[159370.1]
[221315.78]
[178244.23]
[178596.4]
[426422.2]
[194293.4]
[146241.58]
[186041.6]
[180702.28]
[215476.6]
[195470.66]
[159847.22]
[57818.242]
[462325.44]
[340004.12]
[208790.19]
[129623.45]
[223972.28]
[180462.81]
[331046.7]
[245693.]
[230170.]
[161734.69]
[175453.56]
[222648.92]
[140234.2]
[207592.38]
[138970.06]
[326103.62]
[142362.66]
[382283.84]
[173938.83]
[263629.97]
[281881.62]
[171388.47]
[143977.58]
[268945.94]
[33955

[207238.95]
[411285.6]
[112854.5]
[240463.34]
[166425.36]
[168459.1]
[283157.47]
[207089.27]
[99084.195]
[128962.13]
[275835.25]
[288160.88]
[203920.78]
[126175.39]
[143001.02]
[180759.25]
[195117.11]
[285320.97]
[189746.16]
[335774.38]
[171380.45]
[95826.516]
[87523.14]
[743740.9]
[207215.2]
[188594.38]
[272059.28]
[230717.47]
[162983.45]
[144223.4]
[145749.69]
[128345.36]
[225809.02]
[240880.4]
[104939.31]
[661302.9]
[247146.97]
[213162.34]
[228418.38]
[277385.44]
[275610.2]
[164582.42]
[272120.25]
[148567.47]
[120373.67]
[224761.53]
[116833.15]
[226145.81]
[121219.2]
[240949.]
[210613.94]
[222748.88]
[218376.44]
[243873.72]
[199503.81]
[290480.]
[143174.66]
[268849.94]
[142598.28]
[211671.12]
[208536.42]
[181431.88]
[110546.45]
[245840.39]
[306438.53]
[131041.61]
[86217.71]
[187991.28]
[177852.94]
[654631.94]
[140530.28]
[195442.34]
[105719.234]
[371198.97]
[82556.39]
[109407.7]
[154007.22]
[161199.88]
[244128.4]
[294800.9]
[143877.08]
[213002.81]
[372223.16]
[129559.79]
[195080.81]

[223480.16]
[182420.81]
[182461.66]
[130442.164]
[245052.2]
[238900.78]
[140518.97]
[185806.58]
[152380.31]
[316269.8]
[360321.22]
[106910.01]
[198146.]
[248670.94]
[173819.34]
[281279.1]
[131020.2]
[138874.38]
[257848.84]
[186612.19]
[284180.47]
[360848.34]
[60351.94]
[318313.88]
[173650.4]
[140792.83]
[90958.35]
[94220.92]
[160868.1]
[248101.42]
[255735.6]
[131881.17]
[196129.6]
[233426.25]
[229802.17]
[149644.97]
[247095.28]
[415290.84]
[205334.66]
[205452.55]
[184101.47]
[212016.42]
[121335.16]
[165477.05]
[206792.67]
[199552.95]
[94741.11]
[210188.23]
[162844.16]
[271719.3]
[143624.81]
[285284.25]
[336292.56]
[274072.88]
[299944.84]
[268265.38]
[337318.2]
[169357.66]
[234334.22]
[143174.31]
[216788.27]
[136812.34]
[117355.305]
[149813.48]
[168041.42]
[127830.3]
[124145.11]
[159023.12]
[275651.97]
[233446.22]
[277747.38]
[218870.92]
[215023.56]
[189797.28]
[153341.77]
[141651.19]
[308580.4]
[227514.38]
[413912.8]
[262590.6]
[174243.7]
[198065.31]
[162382.34]
[144346.53]
[279601.22]

[272480.4]
[77805.84]
[80983.91]
[226697.19]
[247625.]
[204167.]
[234249.28]
[158970.19]
[152851.03]
[125731.91]
[285270.4]
[85354.64]
[194375.25]
[153485.28]
[280795.]
[160464.72]
[106004.336]
[268886.56]
[150415.27]
[261603.38]
[126129.03]
[173478.75]
[257973.94]
[344566.]
[195610.25]
[119492.35]
[191387.89]
[164649.5]
[99535.57]
[189719.03]
[201179.44]
[122306.81]
[411199.75]
[153651.53]
[138409.97]
[143830.72]
[171363.89]
[216127.06]
[280411.38]
[259087.58]
[178919.28]
[83548.625]
[279012.]
[473946.2]
[251274.78]
[348987.28]
[334003.44]
[255837.34]
[139880.22]
[259804.33]
[454695.28]
[58314.004]
[286589.3]
[208060.67]
[243321.4]
[247931.06]
[157306.7]
[132022.28]
[270046.4]
[210991.56]
[140120.4]
[133182.84]
[116596.25]
[107049.92]
[269913.38]
[359236.25]
[221275.84]
[264210.2]
[106703.49]
[194630.25]
[322903.56]
[211158.52]
[169258.06]
[251761.22]
[319289.25]
[84999.6]
[357356.94]
[156636.66]
[408600.75]
[43207.305]
[175619.95]
[332755.47]
[255900.47]
[193097.73]
[273614.03]
[4632

[152810.81]
[204740.16]
[126132.06]
[123756.414]
[203250.94]
[214104.47]
[172178.73]
[175563.42]
[174518.02]
[106308.82]
[151485.4]
[208103.64]
[73870.93]
[130259.05]
[233885.05]
[239110.22]
[125530.734]
[288013.97]
[177517.38]
[118078.44]
[280911.88]
[282443.2]
[406257.8]
[280243.53]
[140432.88]
[240817.7]
[226481.73]
[254912.36]
[277651.94]
[190496.47]
[123751.14]
[188150.75]
[165447.95]
[254996.06]
[268947.53]
[112236.44]
[262519.8]
[135750.48]
[226409.6]
[117604.94]
[237950.42]
[243595.42]
[190374.44]
[254497.56]
[233889.84]
[302373.47]
[375348.2]
[109410.555]
[274475.6]
[110929.72]
[416869.06]
[191022.4]
[103191.02]
[465064.16]
[123934.375]
[112911.33]
[210873.56]
[119568.15]
[219335.78]
[158216.62]
[319546.47]
[583766.44]
[177939.08]
[108621.34]
[248966.22]
[231295.62]
[182752.03]
[150335.17]
[115353.45]
[202213.16]
[264183.75]
[246266.95]
[147949.25]
[243334.1]
[230674.16]
[382445.25]
[282268.16]
[174623.88]
[199258.2]
[184994.97]
[586090.25]
[168553.17]
[190955.95]
[178298.28]


[458082.53]
[131262.38]
[135757.9]
[199312.72]
[74991.61]
[115151.25]
[274200.06]
[107758.336]
[259229.44]
[106933.41]
[120185.35]
[289102.3]
[248778.42]
[144957.44]
[208962.5]
[231312.72]
[279372.16]
[99231.64]
[261277.53]
[104640.88]
[195877.6]
[177731.16]
[156879.95]
[95283.04]
[113451.984]
[76467.484]
[232353.]
[328710.28]
[155888.94]
[185044.34]
[136509.34]
[293251.22]
[165518.78]
[157008.3]
[206637.95]
[210132.75]
[615935.2]
[179490.72]
[160132.6]
[176099.66]
[78723.49]
[212394.22]
[149853.38]
[208264.06]
[503725.]
[249523.69]
[646504.9]
[188466.05]
[73279.5]
[90763.03]
[73449.15]
[208251.53]
[178110.69]
[231314.11]
[355069.1]
[211564.72]
[221205.34]
[115511.45]
[138684.27]
[147156.66]
[216382.86]
[379154.44]
[310635.5]
[427530.44]
[255618.9]
[256360.6]
[129940.92]
[381763.7]
[129426.78]
[280201.8]
[219750.66]
[207457.78]
[205434.12]
[129128.336]
[363919.5]
[129558.84]
[210541.47]
[124075.36]
[118371.67]
[241330.12]
[152441.1]
[192248.28]
[95086.11]
[410236.5]
[188161.06]
[178040

[85914.766]
[249522.4]
[246814.36]
[92901.85]
[141347.81]
[139926.56]
[60703.01]
[259556.33]
[174542.56]
[227516.61]
[135295.97]
[193137.3]
[202211.6]
[269209.94]
[226995.83]
[250676.9]
[152313.16]
[321338.75]
[351001.56]
[327466.53]
[212235.97]
[243494.9]
[285297.62]
[256343.9]
[135103.77]
[200540.12]
[129752.766]
[200712.84]
[116029.85]
[193371.25]
[152496.05]
[143555.62]
[98231.36]
[404661.8]
[214510.6]
[225140.95]
[233226.94]
[311014.2]
[161907.94]
[176796.05]
[172997.53]
[315519.47]
[126687.69]
[140621.72]
[198562.22]
[160558.34]
[127384.33]
[199418.25]
[347647.44]
[135558.8]
[265258.97]
[461842.28]
[269186.]
[244064.66]
[113514.56]
[197197.48]
[119930.8]
[141133.72]
[252133.34]
[156007.89]
[86394.66]
[149752.38]
[115073.36]
[97294.56]
[302102.88]
[222326.22]
[254741.98]
[130384.5]
[230261.14]
[236714.53]
[143738.05]
[168376.75]
[164787.73]
[210215.33]
[241641.95]
[159196.33]
[175064.69]
[116837.984]
[236330.16]
[247502.12]
[314650.8]
[203053.88]
[147169.62]
[268205.3]
[176948.83]

In [174]:
house_raw_data.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [175]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, final_preds)**0.5

77177.3342160244